<a href="https://colab.research.google.com/github/nitrogenlab/GP15_watermassanalysis/blob/main/pyompa_dev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All needed headers entered and matched (still trouble creating "gp15_intermediateanddeep")  
Parsing of BCODMO data  
Thermocline endmember files in new format  
CTD salinity flags 3 and below

In [1]:
!pip install gsw

     |████████████████████████████████| 2.4 MB 5.2 MB/s 


In [2]:
import gsw

In [3]:
!pip uninstall -y pyompa
%cd /content/
!rm -rf pyompa
!git clone https://github.com/nitrogenlab/pyompa
%cd /content/pyompa
!git checkout dev
!git log -1
!pip install .
%cd /content/

%reload_ext autoreload
%autoreload 2
from importlib import reload
import pyompa
reload(pyompa)

/content
Cloning into 'pyompa'...
remote: Enumerating objects: 1138, done.
remote: Counting objects: 100% (446/446), done.
remote: Compressing objects: 100% (297/297), done.
remote: Total 1138 (delta 310), reused 259 (delta 146), pack-reused 692
Receiving objects: 100% (1138/1138), 14.91 MiB | 21.90 MiB/s, done.
Resolving deltas: 100% (796/796), done.
/content/pyompa
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'
commit a2a3dc30be711b9f84d74f00e844ad1473f05216 (HEAD -> dev, origin/dev)
Author: Av Shrikumar <avanti.shrikumar@gmail.com>
Date:   Thu Sep 23 15:18:03 2021 -0500

    bugfix
Processing /content/pyompa
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discuss

<module 'pyompa' from '/usr/local/lib/python3.7/dist-packages/pyompa/__init__.py'>

In [4]:
%reload_ext autoreload
%autoreload 2
from importlib import reload
import pyompa
reload(pyompa)

<module 'pyompa' from '/usr/local/lib/python3.7/dist-packages/pyompa/__init__.py'>

In [5]:
#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Gla6o_YihOCfU5pWGLhFvL-TKm_0aXfQ' -O names_added_GP15OMPA_33RR20180918_only_gs_rosette_clean1_hy1.csv
!wget http://optserv1.whoi.edu/jgofsopt/80/128.12.123.170/GP15_Bottle_Leg1.mat -O GP15_Bottle_Leg1.mat
!wget http://optserv1.whoi.edu/jgofsopt/80/128.12.123.170/GP15_Bottle_Leg2.mat -O GP15_Bottle_Leg2.mat
#!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Gla6o_YihOCfU5pWGLhFvL-TKm_0aXfQ' -O names_added_GP15OMPA_33RR20180918_only_gs_rosette_clean1_hy1.csv

--2021-10-08 22:51:58--  http://optserv1.whoi.edu/jgofsopt/80/128.12.123.170/GP15_Bottle_Leg1.mat
Resolving optserv1.whoi.edu (optserv1.whoi.edu)... 128.128.181.50
Connecting to optserv1.whoi.edu (optserv1.whoi.edu)|128.128.181.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 934504 (913K) [text/plain]
Saving to: ‘GP15_Bottle_Leg1.mat’

GP15_Bottle_Leg1.ma 100%[===================>] 912.60K  2.72MB/s    in 0.3s    

2021-10-08 22:51:59 (2.72 MB/s) - ‘GP15_Bottle_Leg1.mat’ saved [934504/934504]

--2021-10-08 22:51:59--  http://optserv1.whoi.edu/jgofsopt/80/128.12.123.170/GP15_Bottle_Leg2.mat
Resolving optserv1.whoi.edu (optserv1.whoi.edu)... 128.128.181.50
Connecting to optserv1.whoi.edu (optserv1.whoi.edu)|128.128.181.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1135288 (1.1M) [text/plain]
Saving to: ‘GP15_Bottle_Leg2.mat’

GP15_Bottle_Leg2.ma 100%[===================>]   1.08M  3.49MB/s    in 0.3s    

2021-10-08 22:51:59 (3.49

In [6]:
import scipy.io
leg1 = scipy.io.loadmat('GP15_Bottle_Leg1.mat');
leg2 = scipy.io.loadmat('GP15_Bottle_Leg2.mat');
#merge data sets

In [47]:
#print list of leg1 headers
#list(leg1.keys())

In [48]:
#print list of leg2 headers
#list(leg2.keys())

In [36]:
import pandas as pd
import numpy as np
from collections import OrderedDict

header_mapping = {
    'bottle flag': ('BTLNBR_FLAG_W', 'BTLNBR_FLAG_W'),
    'CTD salinity flag': ('CTDSAL_FLAG_W', 'CTDSAL_FLAG_W'),
    #"bottle salinity flag": ('Flag_SALINITY_D_CONC_BOTTLE_tcj2lg', 'Flag_SALINITY_D_CONC_BOTTLE_zva7jm'),
    "bottle oxygen flag": ('Flag_OXYGEN_D_CONC_BOTTLE_qizf9x', 'Flag_OXYGEN_D_CONC_BOTTLE_n41f8b'),
    "silicate flag": ('Flag_SILICATE_D_CONC_BOTTLE_l9fh07','Flag_SILICATE_D_CONC_BOTTLE_3fot83'),
    "nitrate flag": ('Flag_NITRATE_D_CONC_BOTTLE_xhgtuv','Flag_NITRATE_D_CONC_BOTTLE_bugat8'),
    "phosphate flag": ('Flag_PHOSPHATE_D_CONC_BOTTLE_lof4ap','Flag_PHOSPHATE_D_CONC_BOTTLE_d0rgav'),

    "CTD pressure": ('CTDPRS','CTDPRS'),
    "CTD temperature" : ('CTDTMP','CTDTMP'),
    "practical_salinity" : ('CTDSAL','CTDSAL'), #CTD practical salinity
    "lat" : ('LATITUDE','LATITUDE'),
    "lon": ('LONGITUDE','LONGITUDE'),
    
    "stnnbr": ('STNNBR','STNNBR'),
    "geotrc_ID": ('GEOTRC_SAMPNO','GEOTRC_SAMPNO'),
    "bottom depth": ('DEPTH','DEPTH'),
    "oxygen": ('OXYGEN_D_CONC_BOTTLE_qizf9x','OXYGEN_D_CONC_BOTTLE_n41f8b'),
    "silicate": ('SILICATE_D_CONC_BOTTLE_l9fh07','SILICATE_D_CONC_BOTTLE_3fot83'),
    "nitrate": ('NITRATE_D_CONC_BOTTLE_xhgtuv','NITRATE_D_CONC_BOTTLE_bugat8'),
    'phosphate': ('PHOSPHATE_D_CONC_BOTTLE_lof4ap', 'PHOSPHATE_D_CONC_BOTTLE_d0rgav')
}

dict_for_data_frame = OrderedDict()

def convert_if_string(arr, legname):
  numpy_arr = np.array(arr.squeeze())
  if  (legname == "STNNBR" or legname == "GEOTRC_SAMPNO"):
    return numpy_arr
  elif (str(numpy_arr.dtype) == "<U4" or str(numpy_arr.dtype) == "<U5" 
      or str(numpy_arr.dtype) == "<U6" or str(numpy_arr.dtype) == "<U7" 
      or str(numpy_arr.dtype) == "<U8" or str(numpy_arr.dtype) == "<U9" 
      or str(numpy_arr.dtype) == "<U11" or str(numpy_arr.dtype) == "<U12"): 
      return np.array([(float(x) if x != 'nd' else np.nan) for x in numpy_arr])
  else:
      return numpy_arr

  if (numpy_arr == "<U4" or str(numpy_arr.dtype) == "<U5" 
      or str(numpy_arr.dtype) == "<U6" or str(numpy_arr.dtype) == "<U7" 
      or str(numpy_arr.dtype) == "<U8" or str(numpy_arr.dtype) == "<U9" 
      or str(numpy_arr.dtype) == "<U11" or str(numpy_arr.dtype) == "<U12"): 
      return np.array([(float(x) if x != 'nd' else np.nan) for x in numpy_arr])
  else:
      return numpy_arr

for (new_header_name, (leg1_name, leg2_name)) in header_mapping.items():
  print(new_header_name, leg1_name, leg2_name)
  leg1_arr = convert_if_string(leg1[leg1_name], leg1_name)
  leg2_arr = convert_if_string(leg2[leg2_name], leg2_name)
  print(leg1_arr.dtype)
  if (str(leg1_arr.dtype)=='uint8' or str(leg1_arr.dtype)=='float64'):
    print("leg1 nans", np.sum(np.isnan(leg1_arr)))
    print("leg2 nans", np.sum(np.isnan(leg2_arr)))
  else:
    print('leg1 and leg2 arrays are strings.')
  dict_for_data_frame[new_header_name] = np.concatenate([leg1_arr, leg2_arr])

gp15_df = pd.DataFrame(dict_for_data_frame)

bottle flag BTLNBR_FLAG_W BTLNBR_FLAG_W
uint8
leg1 nans 0
leg2 nans 0
CTD salinity flag CTDSAL_FLAG_W CTDSAL_FLAG_W
uint8
leg1 nans 0
leg2 nans 0
bottle oxygen flag Flag_OXYGEN_D_CONC_BOTTLE_qizf9x Flag_OXYGEN_D_CONC_BOTTLE_n41f8b
uint8
leg1 nans 0
leg2 nans 0
silicate flag Flag_SILICATE_D_CONC_BOTTLE_l9fh07 Flag_SILICATE_D_CONC_BOTTLE_3fot83
uint8
leg1 nans 0
leg2 nans 0
nitrate flag Flag_NITRATE_D_CONC_BOTTLE_xhgtuv Flag_NITRATE_D_CONC_BOTTLE_bugat8
uint8
leg1 nans 0
leg2 nans 0
phosphate flag Flag_PHOSPHATE_D_CONC_BOTTLE_lof4ap Flag_PHOSPHATE_D_CONC_BOTTLE_d0rgav
uint8
leg1 nans 0
leg2 nans 0
CTD pressure CTDPRS CTDPRS
float64
leg1 nans 487
leg2 nans 492
CTD temperature CTDTMP CTDTMP
float64
leg1 nans 487
leg2 nans 492
practical_salinity CTDSAL CTDSAL
float64
leg1 nans 487
leg2 nans 492
lat LATITUDE LATITUDE
float64
leg1 nans 476
leg2 nans 484
lon LONGITUDE LONGITUDE
float64
leg1 nans 476
leg2 nans 484
stnnbr STNNBR STNNBR
<U12
leg1 and leg2 arrays are strings.
geotrc_ID GEOTRC_SAMP

In [37]:
gp15_df

,bottle flag,CTD salinity flag,bottle oxygen flag,silicate flag,nitrate flag,phosphate flag,CTD pressure,CTD temperature,practical_salinity,lat,lon,stnnbr,geotrc_ID,bottom depth,oxygen,silicate,nitrate,phosphate
0,9,9,9,9,9,9,NaN,NaN,NaN,NaN,NaN,Rinse/Test 2,nd,NaN,NaN,NaN,NaN,NaN
1,9,9,9,9,9,9,NaN,NaN,NaN,NaN,NaN,Rinse/Test 2,nd,NaN,NaN,NaN,NaN,NaN
2,9,9,9,9,9,9,NaN,NaN,NaN,NaN,NaN,Rinse/Test 2,nd,NaN,NaN,NaN,NaN,NaN
3,9,9,9,9,9,9,NaN,NaN,NaN,NaN,NaN,Rinse/Test 2,nd,NaN,NaN,NaN,NaN,NaN
4,9,9,9,9,9,9,NaN,NaN,NaN,NaN,NaN,Rinse/Test 2,nd,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6115,2,2,9,2,2,2,1819.7,2.2808,34.6175,-20.0,-151.9991,39.0,15839,4336.0,NaN,109.72,35.30,2.44
6116,2,2,2,2,2,2,1616.9,2.5032,34.5944,-20.0,-151.9991,39.0,15838,4336.0,148.7,101.81,35.25,2.39
6117,2,2,9,2,2,2,1618.0,2.5063,34.5941,-20.0,-151.9991,39.0,15838,4336.0,NaN,101.62,35.30,2.38
6118,2,2,9,2,2,2,1617.0,2.5050,34.5942,-20.0,-151.9991,39.0,15838,4336.0,NaN,101.13,35.23,2.38


In [39]:
import pandas as pd
import numpy as np

def augment_df_with_PO_NO_SiO(df):  
    #remineralization ratios
    r_PO = 155;
    r_SiO = 15
    r_NO = 9.68
    df["PO"] = df["oxygen"] + df["phosphate"]*r_PO
    df["NO"] = df["oxygen"] + df["nitrate"]*r_NO
    df["SiO"] = df["oxygen"] + df["silicate"]*r_SiO
    return df

#remove bad data
for flag_type in ["bottle flag", "CTD salinity flag", "bottle oxygen flag",
                  "silicate flag", "nitrate flag", "phosphate flag"]:
    gp15_df = gp15_df[gp15_df[flag_type] <= 3]
gp15_df = pd.DataFrame(gp15_df)

#create PO offset
#gp15_df["phosphate"] = gp15_df["phosphate"] - 0.2

#add PO and NO to data frame
augment_df_with_PO_NO_SiO(gp15_df)

absolute_salinity = gsw.SA_from_SP(SP=gp15_df["practical_salinity"],
                                   p=gp15_df["CTD pressure"],
                                   lon=gp15_df["lon"],
                                   lat=gp15_df["lat"])
gp15_df["absolute_salinity"] = absolute_salinity

conservative_temp = gsw.CT_from_t(SA=absolute_salinity,
                                  t=gp15_df["CTD temperature"],
                                  p=gp15_df["CTD pressure"])
gp15_df["conservative_temp"] = conservative_temp

potential_temp = gsw.pt_from_CT(SA=absolute_salinity,
                                CT=conservative_temp)
gp15_df["potential_temp"] = potential_temp

sig0 = gsw.rho(SA=absolute_salinity, CT=conservative_temp, p=0) - 1000
gp15_df["sigma0"] = sig0

z = gsw.z_from_p(p=gp15_df["CTD pressure"], lat=gp15_df["lat"])
depth = -z #https://github.com/TEOS-10/python-gsw/blob/7d6ebe8114c5d8b4a64268d36100a70e226afaf6/gsw/gibbs/conversions.py#L577
gp15_df["Depth"] = depth

spic0 = gsw.spiciness0(SA=absolute_salinity, CT=conservative_temp)
gp15_df["spiciness"] = spic0

#calculation of planetary vorticity
rho_ref = 1000.0 #reference density
Omega = 2*np.pi/86400;
f = np.mean(2*Omega*np.sin(gp15_df["lat"]*(np.pi/180.0)))
PV = -(f/rho_ref)*(np.gradient(sig0, depth))
gp15_df["PV"] = PV #potential vorticity

print("Rows:",len(gp15_df))
gp15_df = gp15_df.dropna()
print("Rows without NA values:",len(gp15_df))

Rows: 1144
Rows without NA values: 1143


In [41]:
!wget https://raw.githubusercontent.com/nitrogenlab/GP15_watermassanalysis/main/GP15_mode_waters_intermediateanddeep_endmemberswithsubtypes.csv -O GP15_mode_waters_intermediateanddeep_endmemberswithsubtypes.csv

--2021-10-09 00:34:47--  https://raw.githubusercontent.com/nitrogenlab/GP15_watermassanalysis/main/GP15_mode_waters_intermediateanddeep_endmemberswithsubtypes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4949 (4.8K) [text/plain]
Saving to: ‘GP15_mode_waters_intermediateanddeep_endmemberswithsubtypes.csv’

GP15_mode_waters_in 100%[===================>]   4.83K  --.-KB/s    in 0s      

2021-10-09 00:34:47 (52.4 MB/s) - ‘GP15_mode_waters_intermediateanddeep_endmemberswithsubtypes.csv’ saved [4949/4949]



In [42]:
import pandas
endmember_df = pandas.read_csv("GP15_mode_waters_intermediateanddeep_endmemberswithsubtypes.csv")

#additional_rows = pandas.DataFrame(
#    {"watermass_name": ["PSUW", "ESSW"],
#     "conservative_temp": [4.03, 12.2],
#     "absolute_salinity": [33.38, 34.86],
#     "oxygen": [174.7, 84.6],
#     "phosphate": [1.87, 3.03],
#     "silicate": [21.58, 168.87],
#     "nitrate": [25.87, 43.5]
#    }
#)
endmember_df = augment_df_with_PO_NO_SiO(endmember_df)

In [43]:
endmember_df

,watermass_name,conservative_temp,absolute_salinity,silicate,nitrate,phosphate,oxygen,latitude,longitude,PO,NO,SiO
0,AABW_0,-0.246976,34.813705,120.575318,32.083772,2.219509,233.674298,-64.606559,34.240678,577.698256,544.245206,2042.304061
1,AAIW_0,4.446683,34.407787,20.495798,28.550239,1.997939,231.053341,-50.359275,-88.680001,540.733816,507.419652,538.490310
2,AAIW_1,5.031174,34.384525,11.560925,24.041434,1.671867,272.202930,-53.514262,-84.722780,531.342382,504.924013,445.616811
3,DtW_0,6.541088,32.423083,16.202736,8.900997,1.077766,324.142238,56.173730,-145.280862,491.195969,410.303892,567.183285
4,DtW_1,4.343785,33.368958,42.904067,26.799310,2.040686,235.675219,54.438919,-153.510006,551.981573,495.092542,879.236223
5,ENPCW_0,14.489434,34.641874,5.281252,5.535849,0.483659,218.078556,24.091103,-152.796065,293.045732,271.665572,297.297335
6,ENPCW_1,6.057364,34.182201,58.908883,33.230210,2.412053,102.122309,24.555742,-146.702142,475.990493,423.790742,985.755554
7,ESSW_0,10.491828,34.936362,36.699414,35.446391,2.659579,3.830707,-0.421868,-85.836465,416.065440,346.951770,554.321923
8,ESSW_1,16.294648,35.238711,12.736233,19.757399,1.408791,91.298585,-2.354116,-85.003859,309.661227,282.550209,282.342073
9,EqIW_0,5.470202,34.733371,71.993169,42.428282,3.045580,43.781235,-3.356056,-85.834504,515.846211,454.487000,1123.678774


In [44]:
import json

#gp15_intermediateanddeep = pd.DataFrame(gp15_df[(gp15_df["sigma0"] >= 26.6)])
!wget https://raw.githubusercontent.com/nitrogenlab/GP15_watermassanalysis/main/station_to_tc_cutoffs.json -O station_to_tc_cutoffs.json 
station_to_tcstartend = json.loads(open("station_to_tc_cutoffs.json").read())
gp15_intermediateanddeep = gp15_df[
    gp15_df.apply(lambda x: x['Depth'] > station_to_tcstartend[str(int(x['stnnbr']))]['depth_cutoffs'][1], axis=1)]

--2021-10-09 00:34:55--  https://raw.githubusercontent.com/nitrogenlab/GP15_watermassanalysis/main/station_to_tc_cutoffs.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13389 (13K) [text/plain]
Saving to: ‘station_to_tc_cutoffs.json’

station_to_tc_cutof 100%[===================>]  13.08K  --.-KB/s    in 0s      

2021-10-09 00:34:55 (36.6 MB/s) - ‘station_to_tc_cutoffs.json’ saved [13389/13389]



ValueError: ignored

Visualize the end-members

In [45]:
import matplotlib
from matplotlib import pyplot as plt
import pyompa

matplotlib.rc('axes', labelsize=20)
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20)

endmember_names = endmember_df["watermass_name"]
endmembername_to_indices = pyompa.util.get_endmember_idx_mapping(
                            endmember_names)

colors = ["C0"]*len(endmember_names)
for endmembertypeidx, endmembertype in enumerate(
      endmembername_to_indices.keys()):
  for idx in endmembername_to_indices[endmembertype]:
    colors[idx] = "C"+str(endmembertypeidx)

def make_paired_feature_plot(feature1name, feature2name, xlim=None, ylim=None):
    feature1vals = endmember_df[feature1name]
    feature2vals = endmember_df[feature2name]
    obsdf_feature1vals = gp15_intermediateanddeep[feature1name]
    obsdf_feature2vals = gp15_intermediateanddeep[feature2name]
    
    plt.figure(figsize=(20,10))
    
    ax = plt.gca()
    for i, txt in enumerate(endmember_names):
        plotme = True
        if (xlim is not None and (feature1vals[i] < xlim[0] or feature1vals[i]>xlim[1])):
            plotme = False
        if (ylim is not None and (feature2vals[i] < ylim[0] or feature2vals[i]>ylim[1])):
            plotme = False
        if (plotme):
            ax.annotate(txt, (feature1vals[i], feature2vals[i]), size=15)    
    plt.scatter(obsdf_feature1vals, obsdf_feature2vals, color="blue", s=1)
    plt.scatter(feature1vals, feature2vals, color=colors)
    plt.xlabel(feature1name)
    plt.ylabel(feature2name)
    if xlim is not None:
        plt.xlim(xlim[0], xlim[1])
    if ylim is not None:
        plt.ylim(ylim[0], ylim[1])
    plt.show()

#if you want to zoom into a region...


make_paired_feature_plot("conservative_temp", "absolute_salinity")
make_paired_feature_plot("conservative_temp", "absolute_salinity",
                         xlim=(-0.25, 3), ylim=(34.5,35))

make_paired_feature_plot("conservative_temp", "silicate")
make_paired_feature_plot("conservative_temp", "nitrate")

make_paired_feature_plot("conservative_temp", "phosphate")
make_paired_feature_plot("conservative_temp", "phosphate",
                         xlim=(-0.25, 3), ylim=(2,3))

make_paired_feature_plot("conservative_temp", "oxygen")

matplotlib.rcParams.update(matplotlib.rcParamsDefault)


NameError: ignored

In [ ]:
from collections import OrderedDict

intermediateanddeep_settingtosoln = OrderedDict()

In [ ]:
import pyompa
reload(pyompa)
from pyompa import OMPAProblem, EndMemExpPenaltyFunc
from collections import OrderedDict
import json


convertedparamgroups_staticremineralization = [
    pyompa.ConvertedParamGroup(
        groupname="phosphate_remin",
        conversion_ratios=[{"oxygen": -155, "phosphate": 1.0, "nitrate": 16.0}],
        always_positive=False)
]

convertedparamgroups_flexibleremineralization = [
    pyompa.ConvertedParamGroup(
        groupname="phosphate_remin",
        conversion_ratios=[#Representing C:P = 66
                           {"oxygen": -155*(66.0/106.0),
                            "phosphate": 1.0,
                            "nitrate": 16.0*(66.0/106.0)},
                           #Representing C:P = 209
                           {"oxygen": -155*(209.0/106.0),
                            "phosphate": 1.0,
                            "nitrate": 16.0*(209.0/106.0)}
                          ],
        always_positive=False)
]

rian_weighting = {
    "conservative_temp": 56.0,
    "absolute_salinity": 80.0,
    "silicate": 3.0,
    "nitrate": 5.0,
    "phosphate": 5.0,
    "oxygen": 1.0,
    "NO": 1.0,
    "PO": 0.5
}

withconversion_paramnames = ["conservative_temp", "absolute_salinity",
                             "silicate", "nitrate", "phosphate", "oxygen"]
noconversion_paramnames = ["conservative_temp", "absolute_salinity",
                           "silicate", "NO", "PO"]  

flexiblereminratio_settings = {
    "param_names": withconversion_paramnames,
    "param_weightings": rian_weighting,
    "convertedparam_groups": convertedparamgroups_flexibleremineralization}

staticreminratio_settings = {
    "param_names": withconversion_paramnames,
    "param_weightings": rian_weighting,
    "convertedparam_groups": convertedparamgroups_staticremineralization}

noremin_settings = {
    "param_names": noconversion_paramnames,
    "param_weightings": rian_weighting,
    "convertedparam_groups": []}

endmember_usagepenalty = { 
    "ENPCW_0": EndMemExpPenaltyFunc(
        spec={
            'sigma0':{'type':'density_default', 'upperbound':27.0},
            'lat': {'type':'latlon_default', 'lowerbound':0}
            }),
    "ENPCW_1": EndMemExpPenaltyFunc(
        spec={
            'sigma0':{'type':'density_default', 'upperbound':27.0},
            'lat': {'type':'latlon_default', 'lowerbound':0}
            }),

    "SPCW_0": EndMemExpPenaltyFunc(
        spec={
            'lat': {'type':'latlon_default', 'upperbound':0}
            }),
    "SPCW_1": EndMemExpPenaltyFunc(
        spec={
            'lat': {'type':'latlon_default', 'upperbound':0}
            }),

    "PSUW_0": EndMemExpPenaltyFunc(
        spec={'lat': {'type':'latlon_default', 'lowerbound':10}}),
    "PSUW_1": EndMemExpPenaltyFunc(
        spec={'lat': {'type':'latlon_default', 'lowerbound':10}}),

    "ESSW_0": EndMemExpPenaltyFunc(
        spec={'lat': {'type':'latlon_default', 'lowerbound':-25, 'upperbound':25}}),
    "ESSW_1": EndMemExpPenaltyFunc(
        spec={'lat': {'type':'latlon_default', 'lowerbound':-25, 'upperbound':25}}),

    "EqIW_0": EndMemExpPenaltyFunc(
        spec={'lat': {'type':'latlon_default', 'upperbound':20}}),
    "EqIW_1": EndMemExpPenaltyFunc(
        spec={'lat': {'type':'latlon_default', 'upperbound':20}}),
    
    "LCDW_0": EndMemExpPenaltyFunc(
        spec={'sigma0': {'type':'density_default', 'lowerbound':27.7}}),
    "LCDW_1": EndMemExpPenaltyFunc(
        spec={'sigma0': {'type':'density_default', 'lowerbound':27.7}}),

    "PDW_0": EndMemExpPenaltyFunc(
        spec={'sigma0': {'type':'density_default', 'lowerbound':27.3}}),
    "PDW_1": EndMemExpPenaltyFunc(
        spec={'sigma0': {'type':'density_default', 'lowerbound':27.3}}),


    "AAIW_0": EndMemExpPenaltyFunc(
        spec={'lat': {'type':'latlon_default', 'upperbound':10}}),
    "AAIW_1": EndMemExpPenaltyFunc(
        spec={'lat': {'type':'latlon_default', 'upperbound':10}}),
    
    "UCDW_0": EndMemExpPenaltyFunc(
        spec={'sigma0': {'type':'density_default', 'lowerbound':27.3}}),

    "AABW_0": EndMemExpPenaltyFunc(
        spec={'sigma0': {'type':'density_default', 'lowerbound':27.8},
              'lat': {'type': 'latlon_default', 'upperbound': 30} })
}


for setting_name, setting, endmember_df_touse, usagepenalty_touse in [
    ("flexiblereminratio_manyarchetypes", flexiblereminratio_settings, endmember_df, endmember_usagepenalty),
    
                              #("staticreminratio", staticreminratio_settings),
                              #("noremin_settings", noremin_settings)
                              ]:
    print("On setting:", setting_name)
    ompa_soln = OMPAProblem(
              obs_df=gp15_intermediateanddeep,
              endmembername_to_usagepenaltyfunc=usagepenalty_touse,
              **setting).solve(
                  endmember_df_touse,
                  endmember_name_column="watermass_name",
                  batch_size=100)
    intermediateanddeep_settingtosoln[setting_name] = ompa_soln

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
reload(pyompa)
from pyompa import (plot_ompasoln_endmember_fractions,                       
                    plot_ompasoln_residuals,                                 
                    plot_ompasoln_endmember_usagepenalties)


print("Plotting endmember fractions")
for setting in intermediateanddeep_settingtosoln:
    print("Setting:", setting)
    ompa_soln = intermediateanddeep_settingtosoln[setting]
    plot_ompasoln_endmember_fractions(ompa_soln=ompa_soln,
                                      xaxis_colname="lat",
                                      yaxis_colname="Depth",
                                      group_endmembers=True)
    plot_ompasoln_endmember_fractions(ompa_soln=ompa_soln,
                                      xaxis_colname="lat",
                                      yaxis_colname="Depth",
                                      group_endmembers=False)


print("Plotting absolute residuals")
for setting in intermediateanddeep_settingtosoln:
    print("Setting:", setting)
    ompa_soln = intermediateanddeep_settingtosoln[setting]
    plot_ompasoln_residuals(ompa_soln=ompa_soln,
                            xaxis_colname="lat",
                            yaxis_colname="Depth")
    
print("Plotting penalties")
for setting in intermediateanddeep_settingtosoln:
    print("Setting:", setting)
    ompa_soln = intermediateanddeep_settingtosoln[setting]
    plot_ompasoln_endmember_usagepenalties(ompa_soln=ompa_soln,
                            xaxis_colname="lat",
                            yaxis_colname="Depth")

In [ ]:
INTERANDDEEP_SETTINGTOUSE = "flexiblereminratio_manyarchetypes"

### Intermediate and deep waters uncertainty analysis

In [ ]:
#quantify variation in min/max for each endmember and the remineralization amount
from mpl_toolkits.axes_grid1 import make_axes_locatable
reload(pyompa)
from pyompa.util import collapse_endmembers_by_idxmapping, organize_converted_vars_by_groupname
from pyompa.plotting import plot_residuals, plot_endmember_fractions

soln = intermediateanddeep_settingtosoln[INTERANDDEEP_SETTINGTOUSE]
#soln = intermediateanddeep_settingtosoln["flexiblereminratio_onearchetype"]

#max_resids = np.array([0.01, 0.04, 4, 1, 0.15, 2])
max_resids = np.array([0.001, 0.004, 0.4, 0.1, 0.015, 0.2])

interanddeep_ambiguity_results_max = {}
interanddeep_ambiguity_results_min = {}

for endmemberoverallidx, (endmembername, endmember_idxs) in enumerate(soln.endmembername_to_indices.items()):
    if (True):#endmembername in ["ESSW"]):
        print("On",endmembername)
        maximizing_obj = np.zeros((len(soln.endmember_names) + soln.ompa_problem.num_converted_variables,))
        maximizing_obj[endmember_idxs] = -1
        
        ompasoln_max = pyompa.ompacore.OMPASoln.core_quantify_ambiguity_via_residual_limits(
                                      self=soln, obj_weights=maximizing_obj,
                                      max_resids=max_resids)
        interanddeep_ambiguity_results_max[endmembername] = ompasoln_max
        
        minimizing_obj = np.zeros((len(soln.endmember_names) + soln.ompa_problem.num_converted_variables,))
        minimizing_obj[endmember_idxs] = 1
        ompasoln_min = pyompa.ompacore.OMPASoln.core_quantify_ambiguity_via_residual_limits(
                                      self=soln, obj_weights=minimizing_obj, max_resids=max_resids)
        interanddeep_ambiguity_results_min[endmembername] = ompasoln_min
        

        """print("Original")
        plot_ompasoln_endmember_fractions(ompa_soln=soln,
                                      xaxis_colname="lat",
                                      yaxis_colname="Depth",
                                      group_endmembers=True)
                
        print("With min fractions")
        plot_ompasoln_endmember_fractions(ompa_soln=ompasoln_min,
                                      xaxis_colname="lat",
                                      yaxis_colname="Depth",
                                      group_endmembers=True)
        print("With max fractions")
        plot_ompasoln_endmember_fractions(ompa_soln=ompasoln_max,
                                      xaxis_colname="lat",
                                      yaxis_colname="Depth",
                                      group_endmembers=True)"""
        
        """print("original")
        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(25,5))
        origscatter = ax[0].scatter(soln.obs_df["lat"], -soln.obs_df["Depth"],
                                    c=collapsed_fractions[:,endmemberoverallidx],
                                    vmin=0, vmax=1.0)
        divider = make_axes_locatable(ax[0])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(origscatter, cax=cax)
        ax[0].set_title("Original")
        ax[0].set_ylabel("-Depth")

        lowerscatter = ax[1].scatter(soln.obs_df["lat"], -soln.obs_df["Depth"],
                       c=minwm, #- collapsed_fractions[:,endmemberoverallidx]
                       vmin=0, vmax=1.0)
        divider = make_axes_locatable(ax[1])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(lowerscatter, cax=cax)
        ax[1].set_title("Lowerbound")

        upperscatter = ax[2].scatter(soln.obs_df["lat"], -soln.obs_df["Depth"],
                               c=-maxwm, #- collapsed_fractions[:,endmemberoverallidx]
                               vmin=0, vmax=1.0)
        divider = make_axes_locatable(ax[2])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(upperscatter, cax=cax)
        ax[2].set_title("Upperbound")

        plt.show()"""

## Thermocline Analysis

In [ ]:
#Thermocline files include endmembers in the range of potential density 21 to 27 
#conservative temeprature added in place of PO column
 
!wget https://github.com/nitrogenlab/GP15_watermassanalysis/raw/main/ENPCW.csv -O ENPCW.csv
!wget https://github.com/nitrogenlab/GP15_watermassanalysis/raw/main/ESPCW.csv -O ESPCW.csv
!wget https://github.com/nitrogenlab/GP15_watermassanalysis/raw/main/PSUW.csv -O PSUW.csv
!wget https://github.com/nitrogenlab/GP15_watermassanalysis/raw/main/ESSW.csv -O ESSW.csv
!wget https://github.com/nitrogenlab/GP15_watermassanalysis/raw/main/WSPCW.csv -O WSPCW.csv

In [ ]:
print("First ten rows")
!head "PSUW.csv"
print("last ten rows")
!tail "PSUW.csv"

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt


def read_in_thermocline_endmemberdfs(endmemname_to_csv):
    #This function reads in the data frames for the thermocline end members
    #It takes endmemname_to_csv as input, which maps the thermocline
    # end member name (e.g. ENPCW) to the corresponding csv file.
    endmemname_to_df = {}
    #iterate through every endmemname in endmemname_to_csv
    #Order of thermocline endmember file columns: Sig0 pt SP O Si N P PO Pres spic0 vor NO Lat Lon T]);
    #e.g.
    header = ["c"+str(i) for i in range(1,17)] 
    header[0] = "sigma0" #potential density 
    header[1] = "potential_temp"
    header[2] = "practical_salinity"
    header[3] = "oxygen"
    header[4] = "silicate"
    header[5] = "nitrate"
    header[6] = "phosphate"
    header[7] = "CT" 
    header[8] = "CTD pressure"
    header[9] = "latitude"
    header[10] = "longitude"
    header[11] = "CTD temperature"

    TOPLOT = ["conservative_temp", "absolute_salinity",
              "nitrate", "phosphate", "oxygen"]

    #make sure header names are compatible
    # with those used in the prepare_endmember_df function defined earlier
    # for intermediate waters.
    for endmemname in endmemname_to_csv.keys():
        #get the csv file corresponding to the endmemname
        csv = endmemname_to_csv[endmemname]
        #read the csv file into a data frame df
        df = pd.read_csv(csv)
        augment_df_with_PO_NO_SiO(df)
        #Also add a column to the df that just has the end member name
        # repeated for each row. This will come in handy later when
        # pairing rows from different data frames
        #df["watermassname"] = endmemname
        #store df in endmemname_to_df with the key endmemname
        endmemname_to_df[endmemname] = df;

        fig, ax = plt.subplots(nrows=1, ncols=len(TOPLOT),
                         sharey=True,
                         figsize=(len(TOPLOT)*5, 10))
        for idx,feature_name in enumerate(TOPLOT):
          plt.sca(ax[idx])
          plt.xlabel(feature_name)
          plt.scatter(df[feature_name], df["sigma0"])
        
        plt.sca(ax[0])
        plt.ylabel("sigma0")
        plt.ylim(plt.ylim()[1], plt.ylim()[0])

        print("Plotting for "+endmemname)
        plt.show()

    return endmemname_to_df


#To read in thermocline files via a for loop, first create a dictionary that maps the
# end member name (e.g. PSUW) to the paths for the csv files that
# should be used
endmemname_to_csv = {
    "ENPCW": "ENPCW.csv",
    "ESPCW": "ESPCW.csv",
    "PSUW": "PSUW.csv",
    "ESSW": "ESSW.csv",
    "WSPCW": "WSPCW.csv",
}
#Then, call a function that will run a for loop to read in
# the data frames for the thermocline endmembers.
#The function will return a dictionary that maps the
# thermocline end member to the corresponding dataframe
endmemname_to_df = read_in_thermocline_endmemberdfs(endmemname_to_csv)

In [ ]:
!pip uninstall -y pyompa
%cd /content/
!rm -rf pyompa
!git clone https://github.com/nitrogenlab/pyompa
%cd /content/pyompa
!git checkout dev
!git log -1
!pip install .
%cd /content/

%reload_ext autoreload
%autoreload 2
from importlib import reload
import pyompa
reload(pyompa)

In [ ]:
reload(pyompa)
reload(pyompa.thermocline_array)
from pyompa import ThermoclineArrayOMPAProblem
from collections import OrderedDict

#Define thermocline
#By convention, constants are in all caps
#TC_UPPER_BOUND = 26.6
#TC_LOWER_BOUND = 25.5
TC_UPPER_BOUND = 27
TC_LOWER_BOUND = 22
TC_STEP = 0.01
#therm_endmem_to_use = ["PSUW","ENPCW","ESSW", "SPCW"] #thermocline water masses for analysis
# members to use in the analysis 
#gp15_thermocline = gp15_df[(gp15_df["sigma0"] >= TC_LOWER_BOUND)
#                            & (gp15_df["sigma0"] <= TC_UPPER_BOUND)]
gp15_thermocline =  gp15_df[gp15_df.apply(
        lambda x: (x['Depth'] > station_to_tcstartend[str(int(x['stnnbr']))]['depth_cutoffs'][0])
              and (x['Depth'] < station_to_tcstartend[str(int(x['stnnbr']))]['depth_cutoffs'][1]), axis=1)]

rescale = 0.1 #for stabilizing the solver

thermocline_weighting = {
    "conservative_temp": rescale*200,
    "absolute_salinity": rescale*155,
    "silicate": rescale*5.0,
    "nitrate": rescale*50,
    "phosphate": rescale*50,
    "oxygen": rescale*10
}

settings_to_thermoclineomparesults = OrderedDict()

for (t_settingname, t_weighting) in [
          ('thermocline_weighting', thermocline_weighting),
          ]:
    print("setting name", t_settingname)
    settings_to_thermoclineomparesults[(t_settingname)] = ThermoclineArrayOMPAProblem(
         stratification_col="sigma0",
         tc_lower_bound=TC_LOWER_BOUND,
         tc_upper_bound=TC_UPPER_BOUND,
         tc_step=TC_STEP,
         obs_df=gp15_thermocline,
         param_names=withconversion_paramnames,
         convertedparam_groups=convertedparamgroups_flexibleremineralization,
         param_weightings=t_weighting
     ).solve(endmemname_to_df=endmemname_to_df, max_iter=100000, verbose=False)


In [ ]:
reload(pyompa)
from pyompa import plot_ompasoln_endmember_fractions

print("Plotting endmember fractions")
for setting,thermocline_ompa_results in\
  settings_to_thermoclineomparesults.items():
    print(setting)
    plot_ompasoln_endmember_fractions(
      ompa_soln=thermocline_ompa_results, xaxis_colname="lat", yaxis_colname="Depth")

In [ ]:
from pyompa import plot_ompasoln_residuals

print("Plotting parameter residuals")
for setting,thermocline_ompa_results in\
    settings_to_thermoclineomparesults.items():
    print(setting)
    plot_ompasoln_residuals(ompa_soln=thermocline_ompa_results,
                            xaxis_colname="lat", yaxis_colname="Depth")


In [ ]:
THERMOCLINE_SETTINGTOUSE = "thermocline_weighting"

### Thermocline uncertainty analysis

In [ ]:
reload(pyompa.ompacore)
reload(pyompa.thermocline_array)

#uncertainty analysis for the thermocline
soln = settings_to_thermoclineomparesults[THERMOCLINE_SETTINGTOUSE]
max_resids = 10*np.array([0.001, 0.004, 0.4, 0.1, 0.015, 0.2])
#max_resids = np.array([0.001, 0.004, 0.4, 0.1, 0.015, 0.2])

thermocline_ambiguity_results_max = {}
thermocline_ambiguity_results_min = {}

for endmemberoverallidx, (endmembername, endmember_idxs) in enumerate(soln.endmembername_to_indices.items()):
    if (True):#endmembername in ["ESSW"]):
        print("On",endmembername)
        
        ompasoln_max = (pyompa.thermocline_array.ThermoclineArraySoln
                              .with_endmemtype_names_quant_ambig_via_res_lim(
                         self=soln, endmemtypename_to_weight={endmembername: -1},
                         max_resids=max_resids))
        thermocline_ambiguity_results_max[endmembername] = ompasoln_max

        ompasoln_min = (pyompa.thermocline_array.ThermoclineArraySoln
                              .with_endmemtype_names_quant_ambig_via_res_lim(
                         self=soln, endmemtypename_to_weight={endmembername: 1},
                         max_resids=max_resids))
        thermocline_ambiguity_results_min[endmembername] = ompasoln_min
                
        #print("Original")
        #plot_ompasoln_endmember_fractions(ompa_soln=soln,
        #                              xaxis_colname="lat",
        #                              yaxis_colname="Depth",
        #                              group_endmembers=True)
        
        print("With min fractions")
        plot_ompasoln_endmember_fractions(ompa_soln=ompasoln_min,
                                      xaxis_colname="lat",
                                      yaxis_colname="Depth",
                                      group_endmembers=True)
        print("With max fractions")
        plot_ompasoln_endmember_fractions(ompa_soln=ompasoln_max,
                                      xaxis_colname="lat",
                                      yaxis_colname="Depth",
                                      group_endmembers=True)
        
        #plot_residuals(param_residuals=new_resids_max, param_names=soln.param_names,
        #               xaxis_vals=soln.obs_df["lat"], xaxis_label="lat",
        #               yaxis_vals=soln.obs_df["Depth"], yaxis_label="Depth")
        #plot_residuals(param_residuals=new_resids_min, param_names=soln.param_names,
        #               xaxis_vals=soln.obs_df["lat"], xaxis_label="lat",
        #               yaxis_vals=soln.obs_df["Depth"], yaxis_label="Depth")

## Combining the results for intermediate & deep and the thermocline, plot and export

In [ ]:
reload(pyompa)
from collections import OrderedDict

ORIG_COLS_TO_INCLUDE = ["lat", "lon", "Depth", "stnnbr", "geotrc_ID", "sigma0"]

base_intermediateanddeep =\
  intermediateanddeep_settingtosoln[INTERANDDEEP_SETTINGTOUSE]
base_thermocline =\
  settings_to_thermoclineomparesults[THERMOCLINE_SETTINGTOUSE]

base_combined = pyompa.ompacore.ExportToCsvMixin.merge(
  exptocsv1=base_intermediateanddeep, exptocsv2=base_thermocline)
base_combined.export_to_csv(
    csv_output_name="baseresults.csv",
    orig_cols_to_include=ORIG_COLS_TO_INCLUDE)

combined_ambiguity_results_max = OrderedDict()
combined_ambiguity_results_min = OrderedDict()

all_endmembertypes = sorted(set(
    list(interanddeep_ambiguity_results_max.keys())
    +list(thermocline_ambiguity_results_max.keys())))


for endmembername in all_endmembertypes:
  interanddeep_touse_max = base_intermediateanddeep
  interanddeep_touse_min = base_intermediateanddeep
  thermocline_touse_max = base_thermocline
  thermocline_touse_min = base_thermocline
  if endmembername in interanddeep_ambiguity_results_max:
    interanddeep_touse_max = interanddeep_ambiguity_results_max[endmembername]
    interanddeep_touse_min = interanddeep_ambiguity_results_min[endmembername]
  if endmembername in thermocline_ambiguity_results_max:
    thermocline_touse_max = thermocline_ambiguity_results_max[endmembername]
    thermocline_touse_min = thermocline_ambiguity_results_min[endmembername]
  
  combined_ambiguity_results_max[endmembername] =\
    pyompa.ompacore.ExportToCsvMixin.merge(
      exptocsv1=interanddeep_touse_max,
      exptocsv2=thermocline_touse_max)
  combined_ambiguity_results_max[endmembername].export_to_csv(
    csv_output_name="max_"+endmembername+"_results.csv",
    orig_cols_to_include=ORIG_COLS_TO_INCLUDE,
    #don't export the residuals cos they haven't been subject
    # to minimization beyond satisfying the constraints
    export_residuals=False)
  
  combined_ambiguity_results_min[endmembername] =\
    pyompa.ompacore.ExportToCsvMixin.merge(
      exptocsv1=interanddeep_touse_min,
      exptocsv2=thermocline_touse_min)
  combined_ambiguity_results_min[endmembername].export_to_csv(
    csv_output_name="min_"+endmembername+"_results.csv",
    orig_cols_to_include=ORIG_COLS_TO_INCLUDE,
    export_residuals=False)


#print(np.min([x['sig0_cutoffs'][1] for x in station_to_tcstartend.values()]))
#print(np.max([x['sig0_cutoffs'][1] for x in station_to_tcstartend.values()]))

Plotting (warning: this can make the notebook very large, sometimes too large to save)

In [ ]:
print("Base (original) plot")

plot_ompasoln_endmember_fractions(ompa_soln=base_combined,
                                xaxis_colname="lat",
                                yaxis_colname="Depth",
                                group_endmembers=True)
plot_ompasoln_residuals(ompa_soln=base_combined,
                        xaxis_colname="lat",
                        yaxis_colname="Depth")


In [ ]:
#Put in the endmember names in the list to plot the uncertainty for them
for endmembername in ["AABW", "DtW"]:#all_endmembertypes:
  print("Uncertainty for endmembername:",endmembername)
  #print("Original")
  #plot_ompasoln_endmember_fractions(ompa_soln=base_combined,
  #                              xaxis_colname="lat",
  #                              yaxis_colname="Depth",
  #                              group_endmembers=True)
  print("With min fractions")
  plot_ompasoln_endmember_fractions(
      ompa_soln=combined_ambiguity_results_min[endmembername],
                                xaxis_colname="lat",
                                yaxis_colname="Depth",
                                group_endmembers=True)
  print("With max fractions")
  plot_ompasoln_endmember_fractions(
      ompa_soln=combined_ambiguity_results_max[endmembername],
                                xaxis_colname="lat",
                                yaxis_colname="Depth",
                                group_endmembers=True)

Download to hard drive

In [ ]:
from google.colab import files
import glob

!mkdir ompa_results
!mv *results.csv ompa_results
!zip -r ompa_results.zip ompa_results